In [1]:
# INITIALIZE
import pandas as pd
#!/usr/bin/env python3
import os
from dotenv import load_dotenv
import typing
import fmpsdk

# import helper functions
from helper import to_percentage
from helper import discount_terminal_value
from helper import reinvestment_rate

# import all constant
from constant import INCOMESTATEMENT
from constant import BALANCESHEET
from constant import BALANCEGROWTH
from constant import INCOMEGROWTH
from constant import CASHFLOWSTATEMENT
from constant import PERIOD
from constant import INTRINSICVALUATION
from constant import ADVANCED_DCF
from constant import RATIO




# Actual API key is stored in a .env file.  Not good to store API key directly in script.
load_dotenv()
apikey = os.environ.get("apikey")


In [2]:
# Get data from SDK
symbol: str = input("Please enter stock symbol ")
period:str=input("Enter 'quarter' and 'annual'")
period=period.strip().lower()
symbol=symbol.strip().upper()
# financial statement
income_statement=fmpsdk.income_statement(apikey=apikey,period=period,symbol=symbol)
balance_sheet=fmpsdk.balance_sheet_statement(apikey=apikey,period=period,symbol=symbol)
cashflow_statement=fmpsdk.cash_flow_statement(apikey=apikey,period=period,symbol=symbol)
ratio_data=fmpsdk.financial_ratios(apikey=apikey,symbol=symbol,period=period)


# growth data
if period==PERIOD.ANNUAL.value:
    balance_growth=fmpsdk.balance_sheet_statement_growth(apikey=apikey,symbol=symbol)
    income_growth=fmpsdk.income_statement_growth(apikey=apikey,symbol=symbol)



In [3]:
# FOR THOSE NOT IN SDK, EXTRACT USING URL REQUEST
import requests

if period==PERIOD.ANNUAL.value:
    # advanced discounted cash flow data
    resp=requests.get(f'https://financialmodelingprep.com/api/v4/advanced_discounted_cash_flow',
                    params={
                        'symbol':symbol,
                        'apikey':apikey
                        })
    advance_dcf_data=resp.json()
    print(advance_dcf_data)
    advance_dcf_frame=pd.DataFrame(data=advance_dcf_data)
    advance_dcf_frame=advance_dcf_frame.set_index(keys=ADVANCED_DCF.CALENDAR_YEAR.value,drop=True)
    # for capex, please refer to cashflow statement
    if CASHFLOWSTATEMENT.CAPEX.value in advance_dcf_frame.columns:
        advance_dcf_frame=advance_dcf_frame.drop(labels=CASHFLOWSTATEMENT.CAPEX.value,axis='columns')


[{'year': '2027', 'symbol': 'MED', 'revenue': 5652707918, 'revenuePercentage': 28.74, 'ebitda': 766008338, 'ebitdaPercentage': 13.55, 'ebit': 727071992, 'ebitPercentage': 12.86, 'depreciation': 38936346, 'depreciationPercentage': 0.68881, 'totalCash': 728969465, 'totalCashPercentage': 12.9, 'receivables': 5264012.47, 'receivablesPercentage': 0.09312373, 'inventories': 447015940, 'inventoriesPercentage': 7.91, 'payable': 309199843, 'payablePercentage': 5.47, 'capitalExpenditure': -71339332, 'capitalExpenditurePercentage': -1.26, 'price': 62.1, 'beta': 1.228, 'dilutedSharesOutstanding': 11276000, 'costofDebt': 3.99, 'taxRate': 22, 'afterTaxCostOfDebt': 3.11, 'riskFreeRate': 3.99, 'marketRiskPremium': 4.72, 'costOfEquity': 9.79, 'totalDebt': 26051000, 'totalEquity': 700239600, 'totalCapital': 726290600, 'debtWeighting': 3.59, 'equityWeighting': 96.41, 'wacc': 9.55, 'taxRateCash': 21211417, 'ebiat': 572849726, 'ufcf': 508507724, 'sumPvUfcf': 1215568957, 'longTermGrowthRate': 0.5, 'terminal

In [4]:
# FINANCE RATIO
ratio_frame=pd.DataFrame(data=ratio_data)
if period==PERIOD.ANNUAL.value:
    ratio_frame=ratio_frame.set_index(keys=RATIO.CALENDAR_YEAR.value)
elif period==PERIOD.QUARTER.value:
    ratio_frame=ratio_frame.set_index(keys=[RATIO.CALENDAR_YEAR.value,RATIO.PERIOD.value])

In [5]:
# CASHFLOW STATEMENT
cashflow_columns=[
    CASHFLOWSTATEMENT.CALENDAR_YEAR.value,
    CASHFLOWSTATEMENT.CAPEX.value,
    CASHFLOWSTATEMENT.COMMONSTOCK_REPURCHASE.value,
    CASHFLOWSTATEMENT.DIVIDEND_PAID.value,
    CASHFLOWSTATEMENT.DEPRECIATION.value,
    CASHFLOWSTATEMENT.CHANGE_WORKING_CAP.value,
    CASHFLOWSTATEMENT.NET_INCOME.value,
    CASHFLOWSTATEMENT.FREE_CASH_FLOW.value
]

if period==PERIOD.QUARTER.value:
    cashflow_columns.append('period')

cashflow_frame=pd.DataFrame(data=cashflow_statement,columns=cashflow_columns)

if period==PERIOD.ANNUAL.value:
    cashflow_frame=cashflow_frame.set_index(keys=CASHFLOWSTATEMENT.CALENDAR_YEAR.value)
elif period==PERIOD.QUARTER.value:
    cashflow_frame=cashflow_frame.set_index(keys=[CASHFLOWSTATEMENT.CALENDAR_YEAR.value,CASHFLOWSTATEMENT.PERIOD.value])

In [6]:
# BALANCE SHEET
balance_columns=[BALANCESHEET.CALENDAR_YEAR.value,
                BALANCESHEET.LONGTERM_DEBT.value,
                BALANCESHEET.TOTAL_LIABILITIES.value,
                BALANCESHEET.TOTAL_STOCKHOLDEREQUITIES.value]

if period==PERIOD.QUARTER.value:
    balance_columns.append('period')
balance_frame=pd.DataFrame(data=balance_sheet,columns=balance_columns)

# ratio
balance_frame[RATIO.LIABILITY_TO_SHAREHOLDER_EQUITY.value]=balance_frame[BALANCESHEET.TOTAL_LIABILITIES.value]/balance_frame[BALANCESHEET.TOTAL_STOCKHOLDEREQUITIES.value]

# set index
if period==PERIOD.ANNUAL.value:
    balance_frame=balance_frame.set_index(keys=BALANCESHEET.CALENDAR_YEAR.value)
elif period==PERIOD.QUARTER.value:
    balance_frame=balance_frame.set_index(keys=[BALANCESHEET.CALENDAR_YEAR.value, BALANCESHEET.PERIOD.value])


In [7]:
# BALANCE GROWTH
# sheet analysis only happend once a year
if period==PERIOD.ANNUAL.value:
    balance_growth_columns=[BALANCEGROWTH.CALENDAR_YEAR.value,
                            BALANCEGROWTH.RETAINEDEARNINGS_GROWTH.value]
    balance_growth_frame=pd.DataFrame(data=balance_growth,columns=balance_growth_columns)
    balance_growth_frame[BALANCEGROWTH.RETAINEDEARNINGS_GROWTH.value]=balance_growth_frame[BALANCEGROWTH.RETAINEDEARNINGS_GROWTH.value].map(to_percentage)
    balance_growth_frame=balance_growth_frame.set_index(BALANCEGROWTH.CALENDAR_YEAR.value)

In [8]:
# INCOME GROWTH

if period==PERIOD.ANNUAL.value:
    income_growth_columns=[INCOMEGROWTH.CALENDAR_YEAR.value,INCOMEGROWTH.EBITDA_GROWTH.value]
    income_growth_frame=pd.DataFrame(data=income_growth,columns=income_growth_columns)
    income_growth_frame[INCOMEGROWTH.EBITDA_GROWTH.value]=income_growth_frame[INCOMEGROWTH.EBITDA_GROWTH.value].map(to_percentage)
    income_growth_frame=income_growth_frame.set_index(keys=INCOMEGROWTH.CALENDAR_YEAR.value)

    

In [9]:
# INCOME STATEMENT
# from constant import INCOMESTATEMENT


# selecting fields from original data structure and import them into pandas.DataFrame
income_columns=[INCOMESTATEMENT.CALENDAR_YEAR.value,
                INCOMESTATEMENT.GROSSPROFIT_RATIO.value,
                INCOMESTATEMENT.NETINCOME_RATIO.value,
                INCOMESTATEMENT.GROSS_PROFIT.value,
                INCOMESTATEMENT.RD.value,
                INCOMESTATEMENT.SGA_EXPENSE.value,
                INCOMESTATEMENT.DEPRECIATION.value,
                INCOMESTATEMENT.INTEREST_EXPENSE.value,
                INCOMESTATEMENT.OPERATING_INCOME.value,
                INCOMESTATEMENT.TAX.value
                ]
if period==PERIOD.QUARTER.value:
    income_columns.append('period')
income_frame=pd.DataFrame(data=income_statement,columns=income_columns)

# extract necessary ratio from income statement
income_frame[RATIO.RD_GROSSPROFIT_RATIO.value]=(income_frame[INCOMESTATEMENT.RD.value]/income_frame[INCOMESTATEMENT.GROSS_PROFIT.value]).map(to_percentage)
income_frame[RATIO.SGA_GROSSPROFIT_RATIO.value]=(income_frame[INCOMESTATEMENT.SGA_EXPENSE.value]/income_frame[INCOMESTATEMENT.GROSS_PROFIT.value]).map(to_percentage)
income_frame[RATIO.DEPRECIATION_GROSSPROFIT_RATIO.value]=(income_frame[INCOMESTATEMENT.DEPRECIATION.value]/income_frame[INCOMESTATEMENT.GROSS_PROFIT.value]).map(to_percentage)
income_frame[INCOMESTATEMENT.GROSSPROFIT_RATIO.value]=income_frame[INCOMESTATEMENT.GROSSPROFIT_RATIO.value].map(to_percentage)
income_frame[INCOMESTATEMENT.NETINCOME_RATIO.value]=income_frame[INCOMESTATEMENT.NETINCOME_RATIO.value].map(to_percentage)
income_frame[RATIO.INTEREST_EXP_OPINCOME_RATIO.value]=(income_frame[INCOMESTATEMENT.INTEREST_EXPENSE.value]/income_frame[INCOMESTATEMENT.OPERATING_INCOME.value]).map(to_percentage)

income_columns_todrop=[INCOMESTATEMENT.DEPRECIATION.value]
income_frame=income_frame.drop(labels=income_columns_todrop,axis='columns')

if period==PERIOD.ANNUAL.value:
    income_frame=income_frame.set_index(keys=INCOMESTATEMENT.CALENDAR_YEAR.value)
elif period==PERIOD.QUARTER.value:
    income_frame=income_frame.set_index(keys=[INCOMESTATEMENT.CALENDAR_YEAR.value,INCOMESTATEMENT.PERIOD.value])

In [10]:
# RATIO TABLE
joinkeys=['calendarYear','period'] if period==PERIOD.QUARTER.value else ['calendarYear']
table1=pd.merge(left=income_frame,right=balance_frame,left_index=True,right_index=True)
table1=pd.merge(left=table1,right=cashflow_frame,left_index=True,right_index=True)
if period==PERIOD.ANNUAL.value:
    table1=pd.merge(left=table1,right=balance_growth_frame,left_index=True,right_index=True)
    table1=pd.merge(left=table1,right=income_growth_frame,left_index=True,right_index=True)
#ratio
table1[RATIO.DEBT_NETINCOME_RATIO.value]=(table1[BALANCESHEET.LONGTERM_DEBT.value]/table1[CASHFLOWSTATEMENT.NET_INCOME.value]).round(2)
table1[RATIO.CAPEX_GROSSPROFIT_RATIO.value]=(table1[CASHFLOWSTATEMENT.CAPEX.value].abs()/table1[INCOMESTATEMENT.GROSS_PROFIT.value]).map(to_percentage)

table1_columns=[
    INCOMESTATEMENT.GROSSPROFIT_RATIO.value,
    INCOMESTATEMENT.NETINCOME_RATIO.value,
    RATIO.RD_GROSSPROFIT_RATIO.value,
    RATIO.SGA_GROSSPROFIT_RATIO.value,
    RATIO.DEPRECIATION_GROSSPROFIT_RATIO.value,
    RATIO.INTEREST_EXP_OPINCOME_RATIO.value,
    RATIO.LIABILITY_TO_SHAREHOLDER_EQUITY.value,
    RATIO.DEBT_NETINCOME_RATIO.value,
    RATIO.CAPEX_GROSSPROFIT_RATIO.value,
    CASHFLOWSTATEMENT.COMMONSTOCK_REPURCHASE.value,
    CASHFLOWSTATEMENT.DIVIDEND_PAID.value,
]
if period==PERIOD.ANNUAL.value:
    table1_columns.append(BALANCEGROWTH.RETAINEDEARNINGS_GROWTH.value)
    table1_columns.append(INCOMEGROWTH.EBITDA_GROWTH.value)


table1=table1[table1_columns]

table1.style.set_caption(f'Table 1: stats for {symbol}')




,grossProfitRatio,netIncomeRatio,RD_grossProfit_Ratio,SGA_grossProfit_Ratio,Depreciation_grossProfit_Ratio,interestExpense_operatingIncome_Ratio,liability_shareholder ratio,debt_ netIncome_ratio,capex_grossProfit_rario,commonStockRepurchased,dividendsPaid,growthRetainedEarnings,growthEBITDA
calendarYear,,,,,,,,,,,,,
2022,71.34 %,8.98 %,0.00 %,83.79 %,0.96 %,-0.38 %,1.039505,0.140000,1.46 %,-127961000,-71620000,-26.52 %,-12.22 %
2021,73.89 %,10.75 %,0.00 %,80.82 %,0.60 %,0.00 %,0.967295,0.160000,3.03 %,-62088000,-63856000,23.31 %,66.26 %
2020,74.65 %,11.00 %,0.00 %,80.77 %,1.04 %,0.00 %,0.755746,0.070000,0.84 %,-5551000,-53190000,-8.55 %,47.36 %
2019,75.22 %,10.92 %,0.00 %,83.04 %,0.86 %,0.00 %,0.856809,0.130000,1.87 %,-47206000,-35396000,28.51 %,31.82 %
2018,75.83 %,11.14 %,0.00 %,81.82 %,1.17 %,1.89 %,0.552884,0.000000,1.30 %,-30715000,-23160000,26.58 %,74.26 %
2017,75.54 %,9.19 %,0.00 %,82.60 %,1.85 %,1.41 %,0.343964,0.000000,1.42 %,-2540000,-15390000,10.98 %,47.56 %
2016,74.91 %,6.50 %,0.00 %,86.94 %,2.63 %,1.05 %,0.262456,0.040000,1.40 %,-1342000,-11889000,5.49 %,-6.36 %
2015,73.80 %,7.35 %,0.00 %,85.75 %,3.53 %,2.30 %,0.324460,14.720000,1.40 %,-11812000,0,12.34 %,-5.16 %
2014,73.33 %,4.62 %,0.00 %,85.54 %,3.85 %,2.37 %,0.440305,0.010000,3.36 %,-35046000,0,-19.25 %,-32.74 %


## ratio table explain

| ratio                                     | description                                                                                       | bias                                                                                                                                 |
| ----------------------------------------- | ------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------ |
| gross profit ratio                        | How many money a company makes out of gross revenue after subtracting cost of good sold and labor | higher mean that company has more freedom to adjust the price                                                                        |
| net income ratio                          | net income is gross profit subtract SGA, operating expense, interest and tax expense              | consistently above 20 % is good. If net income ratio is below 10% means that company is facing competition                           |
| RD and gross profit ratio                 | it shows whether the company needs to spend a lot of money to keep up with the competition        |                                                                                                                                      |
| SGA and gross profit ratio                | the ratio tells us how quickly the company is able to cut the cost                                | If the company keep the ratio consistent, it tells us that the company is able to quickly cut the cost when the time is bad for them |
| Depreciation and gross profit ratio       | depreciation is expense spread over the years                                                     | company with durable competitive advantage tends to have lower depreciation and gross profit ratio                                   |
| total liability to shareholder equity     | the ratio tells us how many dolloar in debt for each dollor of shareholder equity                 | company with durable competitive advantage tend to keep this ratio around or below one                                               |
| capital expenditure to gross profit ratio | capital expenditure is the expense that company spend to continue its operation                   | company with durable competitive advantage will keep it consistently below 25 % |


<!-- TODO: write an MD file to explain each ratio -->

In [11]:
# DISCOUNT CASH FLOW - GORDON GROWTH MODEL

if period==PERIOD.ANNUAL.value:


   
    table2=pd.merge(left=income_frame,right=cashflow_frame,left_index=True,right_index=True)
    table2=pd.merge(left=table2,right=ratio_frame,left_index=True,right_index=True)
    table2=pd.merge(left=table2,right=balance_frame,left_index=True,right_index=True)
    table2=pd.merge(left=table2,right=advance_dcf_frame,left_index=True,right_index=True)

    # data transform
    table2[ADVANCED_DCF.TAX_RATE.value]=table2[ADVANCED_DCF.TAX_RATE.value]/100
    table2[ADVANCED_DCF.WACC.value]=table2[ADVANCED_DCF.WACC.value]/100
    table2[ADVANCED_DCF.RISK_FREE_RATE.value]=table2[ADVANCED_DCF.RISK_FREE_RATE.value]/100

    # operations
    table2[INTRINSICVALUATION.OPINCOME_AFTER_TAX.value]=table2[ADVANCED_DCF.EBIT.value]*(1-table2[ADVANCED_DCF.TAX_RATE.value])
    table2[INTRINSICVALUATION.NET_CAPEX.value]=table2[CASHFLOWSTATEMENT.CAPEX.value].abs()-table2[CASHFLOWSTATEMENT.DEPRECIATION.value]
    table2[INTRINSICVALUATION.FCFF.value]=table2[INTRINSICVALUATION.OPINCOME_AFTER_TAX.value]-table2[INTRINSICVALUATION.NET_CAPEX.value]-table2[CASHFLOWSTATEMENT.CHANGE_WORKING_CAP.value]
    table2[INTRINSICVALUATION.AUGMENTED_DIVIDEND.value]=table2[CASHFLOWSTATEMENT.DIVIDEND_PAID.value]+table2[CASHFLOWSTATEMENT.COMMONSTOCK_REPURCHASE.value]
    table2[INTRINSICVALUATION.RETENTION_RATIO.value]=(table2[CASHFLOWSTATEMENT.NET_INCOME.value]-table2[INTRINSICVALUATION.AUGMENTED_DIVIDEND.value].abs())/table2[CASHFLOWSTATEMENT.NET_INCOME.value]
    table2[INTRINSICVALUATION.EXPECTED_GROWTH_FCFE.value]=table2[INTRINSICVALUATION.RETENTION_RATIO.value]*table2[RATIO.RETURN_ON_EQUITY.value]
    
    table2[INTRINSICVALUATION.REINVESTMENT_RATE.value]=table2.apply(lambda x: reinvestment_rate(
        operating_income_after_tax=x[INTRINSICVALUATION.OPINCOME_AFTER_TAX.value],
        net_capex=x[INTRINSICVALUATION.NET_CAPEX.value],
        change_in_working_cap=x[CASHFLOWSTATEMENT.CHANGE_WORKING_CAP.value]
        ),axis='columns')
    
    table2[INTRINSICVALUATION.EXPECTED_GROWTH_FCFF.value]=table2[INTRINSICVALUATION.REINVESTMENT_RATE.value]*table2[RATIO.RETURN_ON_CAPITAL.value]
    
    table2[INTRINSICVALUATION.CALCULATE_TERMINAL_VAL.value]=table2.apply(lambda x: discount_terminal_value(
        fcff=x[INTRINSICVALUATION.FCFF.value],
        wacc=x[ADVANCED_DCF.WACC.value],
        risk_free_rate=x[ADVANCED_DCF.RISK_FREE_RATE.value],
        expected_growth_in_n=x[INTRINSICVALUATION.EXPECTED_GROWTH_FCFF.value],
        reinvestment_rate=x[INTRINSICVALUATION.REINVESTMENT_RATE.value]
    ),axis='columns')

    table2[INTRINSICVALUATION.STOCK_INTRINSIC_VAL.value]=table2[INTRINSICVALUATION.CALCULATE_TERMINAL_VAL.value]/table2[ADVANCED_DCF.DILUTED_SHARE_OUTSTANDING.value]
    
    table2_col=[# ADVANCED_DCF.EBIT.value,
                # ADVANCED_DCF.TAX_RATE.value,
                # INTRINSICVALUATION.OPINCOME_AFTER_TAX.value,
                # INTRINSICVALUATION.NET_CAPEX.value,
                # CASHFLOWSTATEMENT.CHANGE_WORKING_CAP.value,
                INTRINSICVALUATION.FCFF.value,

                
                # INTRINSICVALUATION.AUGMENTED_DIVIDEND.value,
                # CASHFLOWSTATEMENT.NET_INCOME.value,
                # INTRINSICVALUATION.RETENTION_RATIO.value,
                # RATIO.RETURN_ON_EQUITY.value,
                # INTRINSICVALUATION.EXPECTED_GROWTH_FCFE.value,
                INTRINSICVALUATION.REINVESTMENT_RATE.value,
                RATIO.RETURN_ON_CAPITAL.value,
                INTRINSICVALUATION.EXPECTED_GROWTH_FCFF.value,
                ADVANCED_DCF.WACC.value,
                ADVANCED_DCF.RISK_FREE_RATE.value,
                INTRINSICVALUATION.CALCULATE_TERMINAL_VAL.value,
                ADVANCED_DCF.DILUTED_SHARE_OUTSTANDING.value,
                INTRINSICVALUATION.STOCK_INTRINSIC_VAL.value
                
                
                ]

    table2=table2[table2_col]
    

table2.style.set_caption(f'Table 2: Intrinsic stock value for {symbol}')

,free cash flow to firm,reinvestment rate,returnOnCapitalEmployed,expected growth in FCFF,wacc,riskFreeRate,terminal value calculated,dilutedSharesOutstanding,intrinsic value per share
2022,116796680.000000,0.189749,1.054113,0.200016,0.095500,0.039900,3249527024.964189,11276000,288.180829
2021,228916980.000000,-0.357205,0.946375,-0.338050,0.095400,0.039900,469584869.022690,11276000,41.644632
2020,73774420.000000,0.254604,0.814398,0.207349,0.095400,0.039900,2253545928.636608,11276000,199.853310
2019,70475700.000000,-0.045576,0.789823,-0.035997,0.095600,0.039900,630669081.073334,11276000,55.930213
2018,50371840.000000,0.000754,0.632990,0.000477,0.095500,0.039900,542144917.672288,11276000,48.079542


In [12]:
discount_terminal_value(2510,0.0676,0.03,0.075,0.3)

69367.70266727956